In [1]:
%%bash
pip install kafka-python

In [2]:
from kafka import KafkaConsumer, KafkaProducer
import json
import re
from datetime import datetime
import nltk
import pickle
from nltk.corpus import stopwords
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from nltk.stem import WordNetLemmatizer

In [3]:
consumer = KafkaConsumer(
    bootstrap_servers='localhost:9092',
    auto_offset_reset='latest',
    value_deserializer=lambda m: json.loads(m.decode('ascii','ignore'))
)

In [4]:
import json
from json import JSONEncoder
import numpy

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)


In [5]:
producer2 = KafkaProducer(bootstrap_servers='localhost:9092',value_serializer=lambda v: json.dumps(v, cls=NumpyArrayEncoder).encode('utf-8'))

In [6]:
consumer.subscribe('data_to_predict')

In [ ]:
counter = 1

with open('classifier', 'rb') as model:
    classifier = pickle.load(model)

for message in consumer:
    
    data = np.asarray(message.value["array"])
    result=classifier.predict(data)
    final_comment = {

    "schema": {
        "type": "struct",
        "optional": False,
        "version": 1,
        "fields": [
            {
                "field": "span_id",
                "type": "string",
                "optional": True
            },
            {
                "field": "movie_id",
                "type": "string",
                "optional": True
            },
            {
                "field": "user_id",
                "type": "string",
                "optional": True
            },
            {
                "field": "comment",
                "type": "string",
                "optional": True
            },
            {
                "field": "status",
                "type": "string",
                "optional": True
            },
            {
                "field": "comment_date",
                "type": "string",
                "optional": True
            }
        ]
    },
    "payload": {
        "span_id": message.value['spanId'],
        "movie_id": message.value['movieId'],
        "user_id": message.value['userId'],
        "comment": message.value["comment"],
        "comment_date": message.value["comment_date"]   
    }

    }
    
    result = int(result[0])
    
    if result==0:
        final_comment['payload']['status'] = 'normal'
        producer2.send('predicted_normal_1',final_comment)
        print("This looks safe")
    elif result==1:
        final_comment['payload']['status'] = 'anomolous'
        producer2.send('predicted_normal_1',final_comment)
        print("This can be Anomolous")
        
    counter = counter + 1


This looks safe
This can be Anomolous
This looks safe
This looks safe
